In [1]:
import pandas as pd
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import linear_model

In [2]:
FILES_PATH = "../data/"

In [3]:
dataset_1_25 = pd.read_excel(FILES_PATH + "Dataset1_25%.xlsx")
dataset_1_50 = pd.read_excel(FILES_PATH + "Dataset1_50%.xlsx")
dataset_1_75 = pd.read_excel(FILES_PATH + "Dataset1_75%.xlsx")
dataset_1_100 = pd.read_excel(FILES_PATH + "Dataset1_100%.xlsx")

In [4]:
all_data = {
    "25%" : dataset_1_25,
    "50%" : dataset_1_50,
    "75%" : dataset_1_75,
    "100%" : dataset_1_100
}

In [5]:
all_features_1 = ["Tarea",	"Glosario",	"Cuestionario",	"Foro",	"Carpeta",	"Recurso",	"URL"]

In [6]:
features_selected = ["Tarea", "Cuestionario", "Carpeta"]

In [19]:
final_result = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1"])

for dataset in all_data.keys():
    evaluation = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1"])
    #80% para entrenamiento y 20% para evaluación
    train_set = all_data[dataset].iloc[:2308]
    test_set  = all_data[dataset].iloc[2308:]

    #preparando la validacion cruzada
    y = train_set.copy().pop("status")
    X = train_set.copy().values
    ss = ShuffleSplit(n_splits=5, test_size=0.20, random_state=0)
    Kfold = {}
    for i, (train_index, test_index) in enumerate(ss.split(X)):
        Kfold[i] = {
            "train" : train_index,
            "test" : test_index
        }

    for i in Kfold.keys():
        train = train_set.iloc[Kfold[i]["train"]]
        test = train_set.iloc[Kfold[i]["test"]]
        model = linear_model.LogisticRegression(max_iter=200)
        model.fit(train[features_selected], train["status"])
        pred = model.predict(test[features_selected])
        print(pred)
        #y_pred_clasificacion = (pred > 0.8).astype(int)
        accuracy = accuracy_score(test["status"], pred)
        precision = precision_score(test["status"], pred)
        recall = recall_score(test["status"],pred)
        f1 = f1_score(test["status"],pred)
        evaluation = pd.concat([evaluation, pd.DataFrame({"accuracy" : accuracy, 
                                                            "precision" : precision, 
                                                            "recall": recall, 
                                                            "f1": f1}, index = [0])], ignore_index=True)

    final_result = pd.concat([final_result, pd.DataFrame({
        "accuracy" : evaluation["accuracy"].mean(),
        "precision" : evaluation["precision"].mean(),
        "recall" : evaluation["recall"].mean(),
        "f1" : evaluation["f1"].mean()
    }, index = [0])], ignore_index =True)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [20]:
final_result.index = all_data.keys()

In [21]:
final_result

,accuracy,precision,recall,f1
25%,0.817316,0.817316,1.0,0.899415
50%,0.817316,0.817316,1.0,0.899415
75%,0.817316,0.817316,1.0,0.899415
100%,0.817316,0.817316,1.0,0.899415


In [18]:
final_result.to_json(FILES_PATH + "GR_METRICAS_1_fs.json")